# SparkBox


## What is SparkBox?

Sparbox aims to facilitate the integration of EuroCalliope outputs into [ENBIOS](https://github.com/LIVENlab/enbios) and allow the integration of prospective analysis. SparkBox allows to:

- Preprocess the Eurocalliope outputs
    - Clean the different technologies included in the study
    - Adapt the units using conversion factors
- Create the input data for enbios
    - Enbios works with a dictionary as an input. This module helps in the creation of this dictionary for EuroCalliope data
- Run enbios using a prospective approach
    - This module updates the background based on the electricity produced in each scenario
    



In [1]:
from ProspectBackground.util.update_experiment import Prospect

In [2]:
import sys
print(sys.executable)

/home/lex/PycharmProjects/SparkBox/venv/bin/python


## Create an instance of the class Prospect

This module works with the support of an excel file. Check [readme](https://github.com/LexPascal/SparkBox#readme)

Pass the path to Calliope's output, basefile, and the brightway's project and database names.  

In [3]:
enbios_mod = Prospect('/home/lex/Downloads/flow_out_sum.csv',
                       '/home/lex/PycharmProjects/SparkBox/ProspectBackground/data/base_file_simplified_v2.xlsx',
                       'Seeds_exp4', 'db_experiments_2')

Project and Database existing...


If the project or database does not exist, you can create one passing the path to the ecoinvent files 

In [ ]:
enbios_mod = Prospect('/home/lex/PycharmProjects/SparkBox/data_sentinel/flow_out_sum.csv',
                       '/home/lex/PycharmProjects/SparkBox/ProspectBackground/data/base_file_simplified_v2.xlsx',
                       'Seeds_exp4', 'NON_EXISTING_DB')

Database NON_EXISTING_DB not in projects. Want to create a new project? (y/n)y


## Preprocess the data

Run the preprocess function. You can pass a boolean "subregions". If it's set as "False", SparkBox will agregate the data at the national level. If it's set as True, it will include subregions

### Warning
The function can returns some warnings:
- It will return a warn if some technologies included in the Calliope data are not included in the basefile
- Warn if some activity codes defined in the basefile do not exist in the database


In some cases these missing information is on porpuse, but this helps in the identification of possible missing information

In [8]:
enbios_mod.preprocess(subregions=False)


/home/lex/PycharmProjects/SparkBox/ProspectBackground/util/preprocess
Adapting input data...
Input checked. Columns look ok


UnboundLocalError: cannot access local variable 'cal_dat' where it is not associated with a value

In [5]:
# You can check all the missing "alias" in enbios (It returns the name+region):
enbios_mod.excluded_techs_and_regions

[]

## Create the input for enbios

This function creates all the information required for enbios to run.
You can pass a "smaller_vers parameter", which allows to reduce the number of scenarios included. This might be interesting for lighter experiments with enbios and the data

The function warns again with inventories not included in the database

The data is stored in a default foler if no path is passed. In this case, it creates a new folder: *PycharmProjects/SparkBox/ProspectBackground/Default/data_enbios.json

In [6]:
enbios_mod.data_for_ENBIOS()

Creating tree following the structure defined in the basefile
Input data for ENBIOS created
/home/lex/PycharmProjects/SparkBox/ProspectBackground/util
/home/lex/PycharmProjects/SparkBox/ProspectBackground/Default/data_enbios.json
Data for enbios saved in /home/lex/PycharmProjects/SparkBox/ProspectBackground/Default/data_enbios.json
Function data_for_ENBIOS executed in 0.04172492027282715 seconds


You can either check the json file in the specific or default folder, or call the following class atribute

In [7]:
enbios_mod.enbios2_data

{'bw_project': 'Seeds_exp4',
 'activities': defaultdict(dict,
             {'wind_onshore__electricity___DEU': {'id': {'code': '81174ec2c20931c1a36f65c654bbd11e'}},
              'wind_onshore__electricity___ESP': {'id': {'code': '81174ec2c20931c1a36f65c654bbd11e'}},
              'wind_offshore__electricity___ESP': {'id': {'code': '46839adbc3cf15621ddbbe687eaa6df1'}},
              'hydro_run_of_river__electricity___ESP': {'id': {'code': 'b870e3d3ddd5b634d016940064b27532'}},
              'hydro_reservoir__electricity___ESP': {'id': {'code': 'c868c4688fbf78f5ca3787ac3d83312b'}},
              'ccgt__electricity___ESP': {'id': {'code': '4833b6352dfe15c95ae46fd280371cd3'}},
              'chp_biofuel_extraction__electricity___LVA': {'id': {'code': '1952fb157a18463b0028629917057bcf'}},
              'open_field_pv__electricity___LVA': {'id': {'code': '206c2e2c30f45d47ea2e9a3701f8ecc5'}},
              'roof_mounted_pv__electricity___ESP': {'id': {'code': '1d7a3591e7a3d033a25e0eaf699bb50b

## Create market for electricity templates

In order to run enbios with the prospective modification, you need to create the template for each market first. 

The following function will create a template for each region in the study **if it already has a market for electricity defined in ecoinvent.

Following the dendrogram structure, you need to pass the key that define the electricity activities, and the units that correspond to that market

In [10]:
enbios_mod.template_electricity('Electricity_generation', Units='kWh')

Template for future market for electicity in ESP has been created
Template for future market for electicity in LVA has been created
Template for future market for electicity in DEU has been created
Function template_electricity executed in 3.5964560508728027 seconds


In [16]:
# Check some market

enbios_mod.default_market['ESP']

[   Amount Location                              Activity name  \
 0       1      ESP                                   FM4E_ESP   
 1       1       PT            wind_onshore__electricity___ESP   
 2       1       PT           wind_offshore__electricity___ESP   
 3       1       PT      hydro_run_of_river__electricity___ESP   
 4       1       PT         hydro_reservoir__electricity___ESP   
 5       1       PT  chp_biofuel_extraction__electricity___ESP   
 6       1       PT         roof_mounted_pv__electricity___ESP   
 7       1       PT   chp_wte_back_pressure__electricity___ESP   
 8       1       PT                  waste_supply__waste___ESP   
 
                        Activity_code  \
 0  Future_market_for_electricity_ESP   
 1   81174ec2c20931c1a36f65c654bbd11e   
 2   46839adbc3cf15621ddbbe687eaa6df1   
 3   b870e3d3ddd5b634d016940064b27532   
 4   c868c4688fbf78f5ca3787ac3d83312b   
 5   1952fb157a18463b0028629917057bcf   
 6   1d7a3591e7a3d033a25e0eaf699bb50b   
 7   fd635

In [8]:
enbios_mod.get_execution_times()

{'preprocess': 0.3198072910308838, 'data_for_ENBIOS': 0.24968862533569336}